In [ ]:
#抓取新聞連結
def getlink(script):
    arry = [] 
    global count
    global pkcount 
    apl = 'apl'
    soup = BeautifulSoup(res.text)
    for a_tag in soup.select('#result h2 a'):
        alink = a_tag.get('href').strip()
        arry.append(alink)
        for elelink in arry:
            num = str(pkcount)
            if pkcount < 10:
                num = str(0) * 4 + num
            elif pkcount < 100:     
                num = str(0) * 3 + num
            elif pkcount < 1000:     
                num = str(0) * 2 + num
            elif pkcount < 10000:     
                num = str(0) * 1 + num
            pk = apl + num
        links = {'link':alink,'area':area,'page':page,'count':count,'pk':pk}
        linkary.append(links)
        count += 1
        pkcount += 1

In [ ]:
import requests
from datetime import *
from bs4 import BeautifulSoup
import time

#12個行政區['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
ary = ['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
links = {'link':'','area':'','page':'','count':'','pk':''}
linkary =[]
    
#系統時間------------------------------
today = date.today()
year = today.year
month = today.month
day = today.day
if month < 10:
    month = "0" + str(month)
if day < 10:
    day = "0" + str(day)
year = str(year)
month = str(month)
day = str(day)
#--------------------------------------

headers={
'Origin':'http://search.appledaily.com.tw',
'Referer':'http://search.appledaily.com.tw/appledaily/search',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36' 
}
pkcount = 1    # pkcount = 1
for area in ary:
    count = 1
    for page in range(1,12):
        payload = {
        'searchMode':'',
        'searchType':'text',
        'ExtFilter':'',
        'sorttype':'1',
        'keyword':area,
        'rangedate':'[20030502 TO'+year+month+day+'999:99'']',
        'totalpage':'',
        'page':page
        }

        res = requests.post('http://search.appledaily.com.tw/appledaily/search',headers=headers,data=payload)
        script = res.text
        getlink(script)

In [ ]:
import pandas
data1 = pandas.DataFrame(linkary)
data1

In [ ]:
#輸出成CSV
data1.to_csv('apl_link.csv',encoding='utf-8')

In [4]:
#讀取CSV檔
import csv
linkary = []
with open('apltst.csv',) as csvfile:   #apl_linktst
    reader = csv.DictReader(csvfile)
    for row in reader:
        linkary.append(row)

In [5]:
# #取得人氣
def getlike(soup):
    if len(soup.select('.urcc .function_icon.clicked')) > 0:
        reele = soup.select('.urcc .function_icon.clicked')[0].text
        rdrx = re.search('(\d+)',reele)
        likes = rdrx.group(1)
        print "1"
    elif len(soup.select('.abdominis #toolbar .function_icon.clicked')) > 0:
        reele = soup.select('.abdominis #toolbar .function_icon.clicked')[0].text
        rdrx = re.search('(\d+)',reele)
        likes = rdrx.group(1)
        print "2"
    else:
        likes = "0"
    return likes

In [6]:
#抓取新聞內文
import requests
import time
import re  # --
from bs4 import BeautifulSoup

headers = { 
'Origin':'http://search.appledaily.com.tw',
'Referer':'http://search.appledaily.com.tw/appledaily/search',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36'
}

for ele in linkary:
    url = ele['link']
    text = ''
    while True:
        try:
            res = requests.get(url,headers=headers)
            res.text
            soup = BeautifulSoup(res.text)
            break
        except:
            time.sleep(5)
            print "wait"
            
    try:
        ele['title'] = soup.select('hgroup #h1')[0].text.strip()
        ele['date'] = soup.select('.gggs time')[0]['datetime']
        for p_tag in soup.select('#summary'):
            text += p_tag.text.strip()
        ele['content'] = text
        ele['likes'] = getlike(soup)

    except:
        print "This link has some problems!"
        print ele['link'], ele['area'], ele['count']

In [7]:
#新聞內文進入DataFrame
import pandas
data2 = pandas.DataFrame(linkary)

# 移除多餘的 column
# del data2['column name']
apl_ok = data2.drop(data2.columns[[0]], axis=1)
apl_ok

,area,content,count,date,likes,link,page,pk,title
0,中正區,（更新：新增影片）太恐怖了！強颱杜鵑橫掃北台灣，許多建築物外牆的招牌不堪強風吹拂搖搖欲墜，北...,32,2015/09/30/,0,http://www.appledaily.com.tw/realtimenews/arti...,4,apl00032,【蘋果衝現場】嚇！ 北市壁磚掉落砸傷路人
1,中正區,落跑名媛吳麗玲的「元大一品苑」金拍第二拍標脫，拍賣底價2億1016萬元，總面積178.98坪...,33,2015/09/30/,0,http://www.appledaily.com.tw/realtimenews/arti...,4,apl00033,溢價5% 「元大一品苑」2.2億標脫
2,中正區,基隆市長林右昌剛剛在臉書轉貼自來水公司的公告，表示因為安和二街及麥金路口的大水管破裂，今晚6...,34,2015/09/29/,0,http://www.appledaily.com.tw/realtimenews/arti...,4,apl00034,基隆水管破裂 部分區域停水至明天中午
3,中正區,,35,2015/09/27/,0,http://www.appledaily.com.tw/appledaily/articl...,4,apl00035,「龍腦」突遭夷平 屋主跪訴沒同意
4,中正區,(更新：新增影片)北市中正區汀州路都更案中，不同意拆除的「龍腦」戶，昨遭無預警拆除。屋主羅進...,36,2015/09/26/,0,http://www.appledaily.com.tw/realtimenews/arti...,4,apl00036,【更新】​龍腦戶遭拆 屋主哽咽跪哭
5,中正區,北市中正區汀州路都更案不同意戶羅先生堅稱房屋位處「龍腦」，拒絕搬遷，連台北市長柯文哲前往開示...,37,2015/09/26/,0,http://www.appledaily.com.tw/realtimenews/arti...,4,apl00037,龍腦被拆 屋主痛哭下跪：大家幫幫我
6,中正區,(更新:新增影片)北市中正區汀州路都更案不同意戶羅先生堅稱房屋位處「龍腦」，拒絕都更，台北市...,38,2015/09/26/,0,http://www.appledaily.com.tw/realtimenews/arti...,4,apl00038,【更新】都更龍腦戶被拆 柯P：無法所有住戶都同意
7,中正區,,39,2015/09/26/,0,http://www.appledaily.com.tw/appledaily/articl...,4,apl00039,新案議價空間加大
8,中正區,,40,2015/09/26/,0,http://www.appledaily.com.tw/appledaily/articl...,4,apl00040,巨流河 享新店溪首排景觀
9,中正區,台北市政府實施鄰里交通環境改善計畫，以中正區的新營里做為示範里，透過標線型人行道劃設、紅黃標...,69,2015/08/31/,0,http://www.appledaily.com.tw/realtimenews/arti...,7,apl00069,改善鄰里交通 柯P：由點而面擴張


In [8]:
#輸出完成品
apl_ok.to_csv('apl1.csv',encoding='utf-8')

In [ ]:
import requests
headers={
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36'
}
res = requests.get('http://www.appledaily.com.tw/appledaily/article/headline/20151104/36879965/applesearch/媚登峰莊雅清姪兒追公車突昏倒',headers=headers)
print res

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text)

In [ ]:
import re
print len(soup.select('.abdominis #toolbar .function_icon.clicked'))>0
# if len(soup.select('.urcc a')) > 0:
#     reele = soup.select('.urcc a')[0].text
#     print "1"
#     rdrx = re.search('(\d+)',reele)
#     likes = rdrx.group(1)
# elif len(soup.select('.fntss a')) > 0:
#     print "2"
#     reele = soup.select('.fntss a')[0].text
#     print reele
#     rdrx = re.search('(\d+)',reele)
#     likes = rdrx.group(0)
# else:
#     likes = "0"
#     print likes

In [ ]:
import re
print soup.select('.function_icon.clicked')[0].text